# Overview of **SlideRule** Functionality

[SlideRule](https://slideruleearth.io/web/) is a web service that provides on-demand customized data products.  The primary way of accessing SlideRule is through its Python client.

```{admonition} Quick links
* SlideRule Documentation: https://slideruleearth.io/web/
* SlideRule GitHub Repository: https://github.com/ICESat2-SlideRule/sliderule
* SlideRule Python Examples GitHub Repository: https://github.com/ICESat2-SlideRule/sliderule-python
```

This notebook gives an overview of the different functionality SlideRule provides.

```{admonition} Learning Objectives
* How to import and configure the SlideRule Python client
* Survey SlideRule's core and advanced functionality
* Where to find documentation on SlideRule's APIs
```

## I. Import and Configure SlideRule

The SlideRule Python client currently consists of seven primary modules.

* __sliderule__ - the core module
* __earthdata__ - functions that access CMR (NASA's Common Metadata Repository), CMR-STAC, and TNM (The National Map, for the 3DEP data hosted by USGS)
* __h5__ - APIs for directly reading HDF5 and NetCD4 data
* __raster__ - APIs for sampling supported raster datasets
* __icesat2__ - APIs for processing ICESat-2 data
* __gedi__ - APIs for processing GEDI data
* __io__ - functions for reading and writing local files with SlideRule results
* __ipysliderule__ - functions for building interactive Jupyter notebooks that interface to SlideRule

These modules can be imported into your environment like so:

In [1]:
from sliderule import sliderule, earthdata, h5, raster, icesat2, gedi

Once those modules are imported, the next thing you'll likely want to do is configure the client with the settings you want to use.  That is done with a call to `sliderule.init()`.  For detailed documentation on what arguments are supported by the initialization function, check out the [api reference page](https://slideruleearth.io/web/rtd/api_reference/sliderule.html#init).

It is not necessary to call `sliderule.init()` in order to start using the client, since the default settings provide a working system.  Nevertheless, it is a good practice to include a call to this function early in your notebook as a placeholder for when different settings are desired.  For instance, if you want to change the verbosity of the client and enable logging to the console, you can do so as shown below.

In [2]:
sliderule.init(verbose=True)

## II. Core Functionality
- A. [Directly read HDF5 and NetCDF4 files](#h5p)
- B. [Subset ATL03 photon cloud data](#atl03sp)
- C. [Generate customized ATL06 elevations](#atl06p)
- D. [Generate customized ATL06 elevations using ATL08 classifications](#atl08)
- E. [Generate customized ATL08 vegetation metrics](#phoreal)
- F. [Sample rasters at points of interest](#raster)
- G. [Subset GEDI L1B, L2A, L4A](#gedi)

<a id='h5p'></a>
### A. Directly read HDF5 and NetCDF files

The **h5** module provides APIs for directly reading HDF5 and NetCDF4 files hosted by NASA in the cloud. [`h5.h5p`](https://slideruleearth.io/web/rtd/api_reference/h5.html#h5p) is the primary method used to directly read data in the cloud.  The [reference page](https://slideruleearth.io/web/rtd/api_reference/h5.html#h5p) for `h5.h5p` provides a description of each of the arguments needed to make the call and the different options available.  


```{tip}
Under-the-hood, the functions in the **h5** module make HTTP requests to SlideRule servers running in us-west-2, and those servers read the requested data from S3 and return the results in an HTTP response back to the client.
```

In the example below, the first 10 latitudes and longitudes are read from an ATL06 granule.  The results are returned in a dictionary of numpy arrays, where each key is the name of the dataset.  If instead of reading just the first 10 values, all the values need to be read, then "numrows" can be set to `h5.ALL_ROWS`. 

In [3]:
asset = "icesat2"
resource="ATL06_20181017222812_02950102_006_02.h5"
datasets = [
    # latitudes
    {"dataset": "/gt1l/land_ice_segments/latitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt1r/land_ice_segments/latitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt2l/land_ice_segments/latitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt2r/land_ice_segments/latitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt3l/land_ice_segments/latitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt3r/land_ice_segments/latitude", "startrow": 0, "numrows": 10},
    # longitudes
    {"dataset": "/gt1l/land_ice_segments/longitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt1r/land_ice_segments/longitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt2l/land_ice_segments/longitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt2r/land_ice_segments/longitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt3l/land_ice_segments/longitude", "startrow": 0, "numrows": 10},
    {"dataset": "/gt3r/land_ice_segments/longitude", "startrow": 0, "numrows": 10}
]

atl06 = h5.h5p(datasets, resource, asset)

In [4]:
atl06

{'/gt2l/land_ice_segments/longitude': array([-106.95560446, -106.9554911 , -106.95551045, -106.95552958,
        -106.95554864, -106.95556768, -106.95558687, -106.95560602,
        -106.95562507, -106.95564448]),
 '/gt1r/land_ice_segments/longitude': array([-106.98659174, -106.98661194, -106.98663208, -106.98665219,
        -106.98667225, -106.98669217, -106.98671199, -106.98673165,
        -106.98675117, -106.9867706 ]),
 '/gt2l/land_ice_segments/latitude': array([27.00235495, 27.00254677, 27.00272693, 27.00290711, 27.00308729,
        27.00326748, 27.00344765, 27.00362783, 27.00380801, 27.00398817]),
 '/gt3l/land_ice_segments/longitude': array([-106.92358939, -106.92360831, -106.92362694, -106.92364578,
        -106.92366553, -106.92368548, -106.92370511, -106.92372443,
        -106.92374335, -106.92376208]),
 '/gt3r/land_ice_segments/latitude': array([27.00538841, 27.00557226, 27.00575378, 27.00593394, 27.00611417,
        27.00629433, 27.00647438, 27.00665448, 27.00683461, 27.00701

<a id='atl03sp'></a>
### B. Subset ATL03 photon cloud data

The [`icesat2.atl03sp`](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl03sp) function makes an ATL03 subsetting request to SlideRule servers and returns a GeoDataFrame of photons.  Documentation for this function can be found in the [API reference](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl03sp).

In the example below, a set of resources is specified via the `poly`, `rgt`, and `cycle` parameters.  Because each granule contains so many photons, it is necessary when making this call to limit the area over which the subsetting request is made, along with the number of granules inside that area. By supplying a GeoJSON file (which is read and processed by the `sliderule.toregion` function into a format usable by SlideRule), the extent of data read in each granule is trimmed.  By supplying a reference ground track (`rgt`), and the cycle number, the number of granules is reduced - in this case to a single granule.

The other parameters in the request are used to specify different aspects of the ATL03 subsetting request.  The `srt` parameters specifies the surface type, which in this case is land. The surface type is used in conjunction with the next parameter - `cnf` which is the confidence level.  A confindence level of high tells SlideRule to only include photons that are highly likely to be surface reflections off of land.  (As a different example, if the `srt` parameter specified land ice and the confidence level was low, then SlideRule would include all photons that had *at least* a low likelihood of being a reflection off of land ice). Lastly, the `len` and `res` parameters specify the length and resolution of the photon segments being returned.  In this case we are asking for 20m segments of photons every 20m.  The length and step size of the segment does not matter so much if it is only photons being returned, but when other processing parameters are supplied (like minimal along track spread of a segment), then it matters more.

Lastly, the call to `icesat2.atl03sp` is made which sends the HTTP request to SlideRule's servers and then waits and accumulates the response from the servers into a GeoDataFrame, with each row representing a single photon.

```{tip}
For the request below there are only ~1K photons returned.  In the actual data, there are roughly ~300K high-confidence photons inside the Grand Mesa region in the selected granule.  The reason so few photons are returned is because the length of the segment is set to 20m and the default along-track-spread required for a valid segment is also 20m (because the default segment length is 40m).  As a result, most segments are filtered out by SlideRule as not being valid.  This happens all the time to me - I get back a lot less data than I expected because I inadvertently changed one parameter without changing another parameter related to it.  In this case, the way to get all of the photons would be to either change the along-track-spread to 40m ("ats": 40), or tell SlideRule to return invalid segments ("pass_invalid": True).  I left it this way below because I wanted to highlight this common issue, and also because the smaller datasets is faster to load into a GeoDataFrame. 
```


In [5]:
# Build Region of Interest
region = sliderule.toregion('grandmesa.geojson')

# Build ATL03 Subsetting Request Parameters
parms = {
    "poly": region["poly"],
    "rgt": 737,
    "cycle": 16,
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_SURFACE_HIGH,
    "len": 20.0,
    "res": 20.0
}

# Make ATL03 Subsetting Request
atl03 = icesat2.atl03sp(parms)

In [6]:
atl03

,rgt,solar_elevation,background_rate,track,segment_id,cycle,segment_dist,sc_orient,atl08_class,height,relief,yapc_score,landcover,distance,quality_ph,atl03_cnf,snowcover,pair,geometry,spot
time,,,,,,,,,,,,,,,,,,,,
2022-08-09 04:05:01.663190528,737,-19.053331,6000.394172,1,215526,16,4.322238e+06,1,4,1964.221191,0.0,0,255,-4.752241,0,4,255,0,POINT (-108.18757 38.83761),6
2022-08-09 04:05:01.663590656,737,-19.053331,6000.394172,1,215526,16,4.322238e+06,1,4,1964.407227,0.0,0,255,-1.916195,0,4,255,0,POINT (-108.18757 38.83764),6
2022-08-09 04:05:01.663790592,737,-19.072472,2255.589879,2,215532,16,4.322358e+06,1,4,1992.014404,0.0,0,255,-6.999181,0,4,255,0,POINT (-108.15103 38.84145),4
2022-08-09 04:05:01.663790592,737,-19.053331,6000.394172,1,215526,16,4.322238e+06,1,4,1964.859985,0.0,0,255,-0.499761,0,4,255,0,POINT (-108.18757 38.83765),6
2022-08-09 04:05:01.663990528,737,-19.053331,6000.394172,1,215526,16,4.322238e+06,1,4,1965.168213,0.0,0,255,0.916676,0,4,255,0,POINT (-108.18757 38.83766),6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-09 04:05:06.777690624,737,-18.884623,6621.979834,3,217230,16,4.356395e+06,1,4,1796.026733,0.0,0,255,7.281042,0,4,255,1,POINT (-108.15067 39.14964),1
2022-08-09 04:05:06.777790464,737,-18.884623,6621.979834,3,217230,16,4.356395e+06,1,4,1795.569092,0.0,0,255,7.994720,0,4,255,1,POINT (-108.15067 39.14965),1
2022-08-09 04:05:06.777790464,737,-18.884623,6621.979834,3,217230,16,4.356395e+06,1,4,1795.770142,0.0,0,255,7.994993,0,4,255,1,POINT (-108.15067 39.14965),1


<a id='atl06p'></a>
### C. Generate custom ATL06 elevations

The [`icesat2.atl06p`](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl06p) function makes an on-demand processing request to SlideRule servers to generate customized ATL06 elevations and return them in a GeoDataFrame.  Documentation for this function can be found in the [API reference](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl06p).

In the example below, a set of resources are implicitly specified via the `poly` parameter.  By only supplying a polygon defining the region of interest, SlideRule will determine which granules intersect the region and then process all them.

```{tip}
Under-the-hood the SlideRule Python client is using the supplied shapefile to make a call to NASA's CMR system to get a list of ATL03 granules that intersect the region of interest.  The list of granules is passed to the SlideRule servers along with geometry in the shapefile.  The SlideRule servers then distribute the processing of each granule across all the available servers and each is responsible for pulling out the photons insides the region of interest and calculating a set of elevations from them.
```
The other parameters in the request all control different aspects of the ATL03 subsetting and ATL06 algorithm running on the SlideRule servers. Note that the length of the ATL03 segment used to generate an ATL06 elevation has been customized to 20m instead of the 40m in the standard product, and the step size has been similarly customized to 10m instead of 20m.

Lastly, the call to `icesat2.atl06p` makes the processing request by sending an HTTP request to SlideRule's servers and then waiting and accumulating the response from the servers into a GeoDataFrame, with each row representing an elevation calculated from a custom ATL03 segment.

In [7]:
# Build Region of Interest
region = sliderule.toregion('grandmesa.geojson')

# Build ATL06 Request Parameters
parms = {
    "poly": region["poly"],
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_SURFACE_HIGH,
    "ats": 7.0,
    "cnt": 10,
    "len": 20.0,
    "res": 10.0,
}

# Make ATL06 Request
atl06 = icesat2.atl06p(parms)

In [8]:
atl06

,pflags,spot,segment_id,w_surface_window_final,distance,dh_fit_dx,rgt,cycle,dh_fit_dy,h_mean,gt,h_sigma,n_fit_photons,rms_misfit,geometry
time,,,,,,,,,,,,,,,
2018-10-16 10:49:21.175923712,0,6,784255,5.893400,1.570828e+07,0.111905,272,1,0.0,1932.573780,10,0.238099,10,0.526231,POINT (-108.05597 39.17015)
2018-10-16 10:49:21.178226688,0,6,784256,3.892215,1.570830e+07,-0.005205,272,1,0.0,1933.155462,10,0.121129,21,0.545944,POINT (-108.05599 39.17002)
2018-10-16 10:49:21.179784960,0,6,784257,3.499944,1.570831e+07,0.062558,272,1,0.0,1933.855075,10,0.134729,31,0.688412,POINT (-108.05601 39.16992)
2018-10-16 10:49:21.181907456,0,6,784257,3.382582,1.570832e+07,0.113617,272,1,0.0,1935.019797,10,0.090643,31,0.491447,POINT (-108.05603 39.16980)
2018-10-16 10:49:21.183406592,0,6,784258,3.383325,1.570833e+07,0.112181,272,1,0.0,1936.090100,10,0.129987,28,0.603474,POINT (-108.05605 39.16971)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-07 18:00:36.382432256,0,5,217202,3.000000,4.355788e+06,-0.000915,1179,18,0.0,3026.024050,50,0.015205,63,0.120393,POINT (-107.82683 39.14888)
2023-03-07 18:00:36.383838976,0,5,217203,3.000000,4.355798e+06,-0.002173,1179,18,0.0,3026.028156,50,0.014089,59,0.108159,POINT (-107.82684 39.14897)
2023-03-07 18:00:36.385241600,0,5,217203,3.000000,4.355808e+06,0.005213,1179,18,0.0,3026.055055,50,0.013608,66,0.110552,POINT (-107.82685 39.14906)


<a id='atl08'></a>
### D. Generate customized ATL06 elevations using ATL08 classifications

The [`icesat2.atl03sp`](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl03sp) and [`icesat2.atl06p`](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl06p) on-demand processing requests can also include the use of ATL08 data to classify and filter the photons returned and used in the ATL06 elevation calculation.  See the [icesat2 module user guide](https://slideruleearth.io/web/rtd/user_guide/ICESat-2.html#atl08-classification) for further documentation.

In the example below, the request to generate custom ATL06 elevations has been modified to use the ATL08 classifier.  The first modification is to drop the minimum required ATL03 confidence level down to CNF_NOT_CONSIDERED which is the lowest confidence of a non-TEP photon.  (This effectively removes ATL03 confidence level filtering by including all non-TEP photons regardless of how the ATL03 processing labelled it).  The second modification is to include the `atl08_class` parameter and  specify `atl08_ground` in the list of labels that should be used.  This causes the ATL06 elevation to only be calculated from photons labelled as ground by the ATL08 classification system.

In [9]:
# Build Region of Interest
region = sliderule.toregion('grandmesa.geojson')

# Build ATL06 Request Parameters
parms = {
    "poly": region["poly"],
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_NOT_CONSIDERED, # effectively remove atl03 filtering
    "ats": 7.0,
    "cnt": 10,
    "len": 20.0,
    "res": 10.0,
    "atl08_class": ["atl08_ground"] # specify ground photons only
}

# Make ATL06 Request
atl06 = icesat2.atl06p(parms)

In [10]:
atl06

,pflags,spot,segment_id,w_surface_window_final,distance,dh_fit_dx,rgt,cycle,dh_fit_dy,h_mean,gt,h_sigma,n_fit_photons,rms_misfit,geometry
time,,,,,,,,,,,,,,,
2018-10-16 10:49:21.178253056,0,6,784256,3.934149,1.570830e+07,-0.008607,272,1,0.0,1933.016317,10,0.102701,17,0.422847,POINT (-108.05599 39.17002)
2018-10-16 10:49:21.179791360,0,6,784257,3.000000,1.570831e+07,0.042971,272,1,0.0,1933.566470,10,0.127884,23,0.528953,POINT (-108.05601 39.16992)
2018-10-16 10:49:21.181908992,0,6,784257,3.000000,1.570832e+07,0.110146,272,1,0.0,1934.745216,10,0.066839,21,0.300498,POINT (-108.05603 39.16980)
2018-10-16 10:49:21.183407872,0,6,784258,3.000000,1.570833e+07,0.104661,272,1,0.0,1935.839538,10,0.101096,18,0.318448,POINT (-108.05605 39.16971)
2018-10-16 10:49:21.184901632,0,6,784258,3.155259,1.570834e+07,0.043212,272,1,0.0,1936.487611,10,0.090417,13,0.324653,POINT (-108.05606 39.16962)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-07 18:00:37.376869376,0,2,217425,3.000000,4.360248e+06,-0.050124,1179,18,0.0,2162.878838,20,0.052424,26,0.219636,POINT (-107.90326 39.18353)
2023-03-07 18:00:37.378268160,0,2,217425,3.062824,4.360258e+06,-0.117692,1179,18,0.0,2161.919798,20,0.109060,11,0.302979,POINT (-107.90327 39.18362)
2023-03-07 18:00:37.389504000,0,2,217429,5.035891,4.360338e+06,-0.196024,1179,18,0.0,2148.707594,20,0.173830,10,0.481328,POINT (-107.90335 39.18434)


<a id='phoreal'></a>
### E. Generate customized ATL08 vegetation metrics

The [`icesat2.atl08p`](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl08p) function makes an on-demand processing request to SlideRule servers to generate customized ATL08 vegetation metrics and return them in a GeoDataFrame.  Documentation for this function can be found in the [API reference](https://slideruleearth.io/web/rtd/api_reference/icesat2.html#atl08p).  

In the example below, vegetation metrics are calculated for every 20m segment in the Grand Mesa region that pass the provided criteria (`ats` - along track spread, `cnt` - minimum number of photons in a segment).  The photons are classified using the ATL08 classification method, and in addition to ground photons, canopy and top of canopy photons are also included (which is necessary for the vegetation statistics).

The set of parameters specific to the ATL08 processing are provided under the `phoreal` key.  The name comes from the University of Texas team that developed PhoREAL and collaborated with us to get their algorithms into SlideRule. Documentation on the different parameters related to the vegetation calculations in PhoREAL can be found in the [user guide](https://slideruleearth.io/web/rtd/user_guide/ICESat-2.html#phoreal-parameters).

Lastly, the call to `icesat2.atl08p` makes the processing request by sending an HTTP request to SlideRule's servers and then waiting and accumulating the response from the servers into a GeoDataFrame, with each row representing a set of vegetation metrics for a custom 20m ATL03 segment.

In [11]:
# Build Region of Interest
region = sliderule.toregion('grandmesa.geojson')

# Build ATL08 Request Parameters
parms = { 
    "poly": region['poly'],
    "cnf": icesat2.CNF_NOT_CONSIDERED,
    "ats": 5.0,
    "cnt": 5,
    "len": 20.0,
    "res": 10.0,
    "atl08_class": [
        "atl08_ground", 
        "atl08_canopy", 
        "atl08_top_of_canopy"
    ],
    "phoreal": {
        "binsize": 1.0, 
        "geoloc": "center", 
        "use_abs_h": False, 
        "send_waveform": False
    }
}

# Make ATL08 Processing Request
atl08 = icesat2.atl08p(parms)

In [12]:
atl08

,landcover,h_te_median,rgt,veg_ph_count,canopy_openness,spot,cycle,canopy_h_metrics,h_max_canopy,h_min_canopy,ph_count,snowcover,solar_elevation,gt,distance,h_canopy,h_mean_canopy,segment_id,gnd_ph_count,geometry
time,,,,,,,,,,,,,,,,,,,,
2018-10-16 10:49:21.175428352,40,1931.684814,272,3,0.120013,6,1,"(1.26806640625, 1.26806640625, 1.26806640625, ...",1.268066,1.003418,10,1,-30.342127,10,1.570827e+07,1.268066,1.098796,784255,7,POINT (-108.05597 39.17018)
2018-10-16 10:49:21.176328192,40,1932.688477,272,0,NaN,6,1,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000,4,1,-30.342127,10,1.570828e+07,0.000000,NaN,784256,4,POINT (-108.05597 39.17013)
2018-10-16 10:49:21.178228224,40,1932.964355,272,4,0.196118,6,1,"(1.0474853515625, 1.0474853515625, 1.047485351...",1.047485,0.605713,21,1,-30.342186,10,1.570829e+07,1.047485,0.816437,784256,17,POINT (-108.05599 39.17002)
2018-10-16 10:49:21.179278080,40,1933.456299,272,8,0.214094,6,1,"(1.25537109375, 1.25537109375, 1.25537109375, ...",1.255371,0.605713,31,1,-30.342186,10,1.570830e+07,1.255371,0.887772,784257,23,POINT (-108.05600 39.16995)
2018-10-16 10:49:21.181978112,30,1935.172119,272,10,0.236772,6,1,"(1.25537109375, 1.25537109375, 1.25537109375, ...",1.255371,0.518311,31,1,-30.342239,10,1.570832e+07,1.255371,0.888391,784257,21,POINT (-108.05603 39.16979)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-07 18:00:37.392222208,126,2146.932617,1179,0,NaN,2,18,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000,11,2,41.786434,20,4.360358e+06,0.000000,NaN,217430,11,POINT (-107.90337 39.18451)
2023-03-07 18:00:37.392222208,126,2146.932617,1179,0,NaN,2,18,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000,11,2,41.786434,20,4.360358e+06,0.000000,NaN,217430,11,POINT (-107.90337 39.18451)
2023-03-07 18:00:37.392222208,126,2146.932617,1179,0,NaN,2,18,"(0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.000000,0.000000,11,2,41.786434,20,4.360358e+06,0.000000,NaN,217430,11,POINT (-107.90337 39.18451)


<a id='raster'></a>
### F. Sample rasters at points of interest

Many of the SlideRule processing APIs support sampling raster datasets at every point generated by the server-side processing.  For a detailed discussion of this capability see the [GeoRaster page](https://slideruleearth.io/web/rtd/user_guide/GeoRaster.html) and the [sampling parameters](https://slideruleearth.io/web/rtd/user_guide/SlideRule.html#raster-sampling) in the SlideRule documentation.

In the example below, an on-demand ATL06 processing request is made along with parameters that specify that the Harmonized Landsat Sentinel-2 (HLS) raster dataset and the GEDI L4B raster dataset is to be sampled at the location of every calculated ATL06 elevation.  The results are returned in a GeoDataFrame where each row contains both an elevation and the value of the sampled HLS and GEDI L4B rasters.

In [13]:
# Build Region of Interest
region = sliderule.toregion('grandmesa.geojson')
catalog = earthdata.stac(short_name="HLS", polygon=region["poly"], time_start="2022-01-01T00:00:00Z", time_end="2022-03-01T00:00:00Z", as_str=True)

# Build Sampling Request Parameters
samples = {
    "landsat": {
        "asset": "landsat-hls",
        "catalog": catalog,
        "closest_time": "2022-01-05T00:00:00Z", 
        "bands": ["NDVI"]
    },
    "gedi": {
        "asset": "gedil4b"
    }   
}

# Build ATL03 Subsetting Request Parameters
parms = {
    "poly": region["poly"],
    "rgt": 737,
    "cycle": 16,
    "srt": icesat2.SRT_LAND,
    "cnf": icesat2.CNF_SURFACE_HIGH,
    "len": 20.0,
    "res": 20.0,
    "samples": samples
}

# Make ATL06 with Sampling Request
atl06 = icesat2.atl06p(parms)

In [14]:
atl06

,pflags,spot,segment_id,w_surface_window_final,distance,dh_fit_dx,rgt,cycle,dh_fit_dy,h_mean,...,rms_misfit,geometry,gedi.value,gedi.file_id,gedi.time,gedi.flags,landsat.value,landsat.file_id,landsat.time,landsat.flags
time,,,,,,,,,,,,,,,,,,,,,
2022-08-09 04:05:02.004134656,0,5,215526,7.207511,4.322238e+06,0.272569,737,16,0.0,1950.969121,...,1.052384,POINT (-108.18666 38.83772),3.732060,0,1.312070e+12,0,0.320962,6,1.325355e+09,0
2022-08-09 04:05:02.007694848,0,3,215533,8.259963,4.322378e+06,0.246685,737,16,0.0,1979.649742,...,1.515434,POINT (-108.15011 38.84176),3.381883,0,1.312070e+12,0,0.393827,6,1.325355e+09,0
2022-08-09 04:05:02.028692992,0,4,215661,15.992866,4.324944e+06,0.008062,737,16,0.0,2345.645139,...,2.194247,POINT (-108.15391 38.86470),5.138969,0,1.312070e+12,0,0.326095,6,1.325355e+09,0
2022-08-09 04:05:02.192992768,0,5,215593,5.802608,4.323581e+06,0.226286,737,16,0.0,1973.237144,...,0.953454,POINT (-108.18809 38.84977),-0.464319,0,1.312070e+12,0,0.163336,6,1.325355e+09,0
2022-08-09 04:05:02.468495104,0,1,215702,10.424640,4.325766e+06,0.122638,737,16,0.0,2392.190077,...,1.382781,POINT (-108.11696 38.87492),8.237352,0,1.312070e+12,0,0.201469,6,1.325355e+09,0
2022-08-09 04:05:03.104293888,0,3,215922,3.000000,4.330176e+06,0.004983,737,16,0.0,3046.448365,...,0.331792,POINT (-108.15837 38.91171),10.128394,0,1.312070e+12,0,-0.032250,6,1.325355e+09,0
2022-08-09 04:05:03.199091456,0,4,216076,30.000000,4.333263e+06,-1.150922,737,16,0.0,2757.357326,...,6.329727,POINT (-108.16288 38.93931),112.406910,0,1.312070e+12,0,0.214478,6,1.325355e+09,0
2022-08-09 04:05:03.968494336,0,1,216234,29.169269,4.336430e+06,0.090866,737,16,0.0,2900.454914,...,3.977454,POINT (-108.12883 38.97054),13.943795,0,1.312070e+12,0,0.277238,6,1.325355e+09,0
2022-08-09 04:05:03.971295232,1,1,216235,19.537112,4.336450e+06,0.209095,737,16,0.0,2902.901898,...,0.000000,POINT (-108.12885 38.97072),13.943800,0,1.312070e+12,0,0.277238,6,1.325355e+09,0


<a id='gedi'></a>
### G. Subset GEDI L1B, L2A, L4A

The [`gedi`](https://slideruleearth.io/web/rtd/api_reference/gedi.html) module provides a handful of functions to make GEDI subsetting requests to SlideRule.  For documentation on those functions, see the [api reference](https://slideruleearth.io/web/rtd/api_reference/gedi.html).  For documentation on the general GEDI capabilities in SlideRule, see the [user guide](https://slideruleearth.io/web/rtd/user_guide/GEDI.html).

In the example below, a request to subset the GEDI L2A data for the Grand Mesa region is made.  The polygon representing the region is provided in the same way it is for the _icesat2_ processing requests. The `*_flag` settings are filters specifying that data is only to be returned for footprints with those flags set that way. The `beam` parameter specifies a single beam number, a list of beam numbers, or all of the beams (`gedi.ALL_BEAMS`).

The call to [`gedi.gedi02ap()`](https://slideruleearth.io/web/rtd/api_reference/gedi.html#gedi02ap) sends the HTTP request to the SlideRule servers and waits for and accumulates the response into a GeoDataFrame where each row represents a returned footprint.

```{tip}
While GEDI L3 and L4 data is all hosted in the cloud, the GEDI L1 and L2 data has not been migrated to the cloud yet and therefore the areas of interest that SlideRule can support are very limited.  To work around this, we have data for a few areas of interest staged in our own S3 bucket.  If you want to use GEDI L1 and/or L2 data for a given area of interest, please get in touch with the SlideRule team and we can work with you to have the data staged in our own bucket until the official products are migrated to the cloud.
```

In [15]:
# Build Region of Interest
region = sliderule.toregion('grandmesa.geojson')

# Build GEDI L2A Request Parameters
parms = {
    "poly": region["poly"],
    "degrade_flag": 0,
    "quality_flag": 1,
    "beam": 0
}

# Make GEDI L2A Request
gedi02a = gedi.gedi02ap(parms)

In [16]:
gedi02a

,solar_elevation,sensitivity,beam,elevation_hr,flags,elevation_lm,geometry
time,,,,,,,
2019-04-19 22:03:34.126039040,42.844566,0.844447,0,2838.221680,128,2833.764404,POINT (-107.79578 39.19455)
2019-04-19 22:03:34.134302976,42.844311,0.970264,0,2851.676514,130,2830.363770,POINT (-107.79527 39.19422)
2019-04-19 22:03:34.142567168,42.844055,0.933131,0,2845.024170,130,2831.502441,POINT (-107.79475 39.19390)
2019-04-19 22:03:34.150831104,42.843800,0.947584,0,2851.234131,130,2840.221924,POINT (-107.79424 39.19358)
2019-04-19 22:03:34.159095040,42.843544,0.863980,0,2862.396973,128,2841.870850,POINT (-107.79373 39.19325)
...,...,...,...,...,...,...,...
2022-11-30 20:35:01.648558336,25.222191,0.964243,0,2456.310547,130,2448.528076,POINT (-107.85977 39.18754)
2022-11-30 20:35:01.656822528,25.221722,0.959823,0,2455.208008,130,2448.547852,POINT (-107.85926 39.18786)
2022-11-30 20:35:01.665086208,25.221254,0.927946,0,2454.385742,130,2450.793701,POINT (-107.85874 39.18818)


## III. Advanced Functionality
- Private clusters
- GeoParquet output to S3
- Customized YAPC classification
- Include ancillary data
- Query CMR, CMR-STAC, and TNM
- Directly sample supported raster datasets
- Subset via rasterized area of interest
- Kmeans clustering